In [12]:
import cv2
import numpy as np
import mediapipe as mp

2024-03-24 10:15:27.367701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 10:15:27.367859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 10:15:27.465665: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 10:15:27.670360: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 10:15:28.777157: W tensorflow/compiler/tf2

In [13]:
# Список путей к изображениям часов
watch_images = [
    "watch1.png",
    # "watch2.png",
    # ... Добавьте больше изображений часов
]

In [14]:
# Загрузка изображений часов
watches = [cv2.imread(path) for path in watch_images]

In [15]:
# Инициализация захвата видео с веб-камеры
cap = cv2.VideoCapture(0)

[ WARN:0@499.849] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@499.850] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range


In [16]:
# Функция для выбора часов (пример с использованием щелчка мыши)
def select_watch(event, x, y, flags, param):
    global selected_watch
    if event == cv2.EVENT_LBUTTONDOWN:  # Пример использования щелчка мыши
        for i, watch in enumerate(watches):
            if y in range(i * 50, (i + 1) * 50):  # Предполагаем, что часы отображаются вертикально
                selected_watch = i
                break

In [17]:
# Инициализация рук медиапайпа
mp_hands = mp.solutions.hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [18]:
# Основной цикл
selected_watch = None

In [20]:
while True:
    ret, frame = cap.read()

    # Проверьте, был ли кадр успешно прочитан
    if ret:
        # Преобразовать кадр в формат RGB для MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Обнаруживать руки
        results = mp_hands.process(frame_rgb)
    
        # Если обнаружена рука
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Получить координаты ориентира на запястье
                wrist_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * frame.shape[1])
                wrist_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * frame.shape[0])
                
                # Нарисуйте круг на запястье для визуализации
                cv2.circle(frame, (wrist_x, wrist_y), 5, (0, 255, 0), -1)
    
                # Наложение часов, если выбрано
                if selected_watch is not None:
                    watch_img = watches[selected_watch]
    
                    # Resize watch based on wrist distance to middle finger (approximate)
                    middle_finger_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * frame.shape[1])
                    wrist_to_finger_dist = abs(wrist_x - middle_finger_x)
                    watch_width = int(wrist_to_finger_dist * 1.5)  # Adjust scaling factor as needed
                    watch_height = int(watch_width * watch_img.shape[0] / watch_img.shape[1])
                    resized_watch = cv2.resize(watch_img, (watch_width, watch_height))
    
                    # Overlay watch using alpha blending
                    alpha = 0.7  # Adjust transparency as needed
                    beta = 1 - alpha
                    overlay_region = frame[wrist_y - watch_height//2:wrist_y + watch_height//2, 
                                           wrist_x - watch_width//2:wrist_x + watch_width//2]
                    frame[wrist_y - watch_height//2:wrist_y + watch_height//2, 
                          wrist_x - watch_width//2:wrist_x + watch_width//2] = cv2.addWeighted(overlay_region, alpha, resized_watch, beta, 0.0)
    
    
        # Отображение результата
        cv2.imshow("Frame", frame)
    
        # Выход из цикла при нажатии клавиши "q"
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        print("Error: Could not capture frame from video stream.")
        break  # Exit the loop if no frame is captured


Error: Could not capture frame from video stream.


In [ ]:
cap.release()
cv2.destroyAllWindows()